<a href="https://colab.research.google.com/github/EX3exp/MiriVoiceSupport-VITS2/blob/main/VITS2_MiriVoice_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Set Up | 환경 설치
#@markdown Please turn on your GPU. <br> Colab Pro is highly recommended, And  THERE'S NO NEED TO USE A100 - if you're willing to use default config file. (L4 IS ENOUGH)
#@markdown <br>반드시 GPU를 활성화해 주세요. <br> Colab Pro를 추천하며, 가속기는 L4로 설정하면 딱 알맞습니다. (내장된 config 파일을 그대로 사용한다는 전제 하에, A100은 굳이 사용하지 않아도 됩니다.)

print(f"🔸 1. Cloning MiriVoiceSupport-VITS2 Github...")
!git clone https://github.com/EX3exp/MiriVoiceSupport-VITS2.git

print(f"✔️ Done!")

!apt-get install espeak


print("\n\n")
print(f"🔸 2. Installing Dependencies...")

!pip install tensorflow[and-cuda]==2.16.1
!pip install -r /content/MiriVoiceSupport-VITS2/requirements.txt



print(f"✔️ Done!")




print("\n\n")
print("✔️✔️✔️ Please Restart Session! | 세션을 다시 시작해 주세요!")


In [ ]:
#@title MiriVoice-VITS2 Voicer Making Notebook <br>미리보이스가 제공하는 VITS2 훈련용 콜랍 노트북이에요.
#@markdown <br>MiriVoice Github ➡️ https://github.com/EX3exp/MiriVoice
#@markdown <br><br> Reference | 참조자료 : <br> https://arxiv.org/abs/2307.16430 <br> https://github.com/p0p4k/vits2_pytorch

#@markdown 📖 [EN] Please select Notebook's language. <br> Messages will be appeared with selected language.
#@markdown <br>📖 [KO] 노트북의 언어를 골라 주세요. <br> 설정된 언어로 노트북의 메시지들이 표시됩니다.
language = "한국어" #@param = ["English", "한국어"]

In [ ]:
#@title 0-1. Mount Google Drive | 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
import os

dir_paths = ["/content/drive/MyDrive/MiriVoice/VITS2/train/dataset", "/content/drive/MyDrive/MiriVoice/VITS2/voicer", "/content/drive/MyDrive/MiriVoice/VITS2/checkpoints"]

train_main_path = "/content/train"

if (language == "English"):
    print(f"✔️ Generated Directory in colab runtime -- {train_main_path}")
elif (language == "한국어"):
    print(f"✔️ 코랩 런타임 내에 폴더를 생성했어요 -- {train_main_path}")

os.makedirs("dir_path", exist_ok=True)
for dir_path in dir_paths:
    os.makedirs(dir_path, exist_ok=True)
    if (language == "English"):
        print(f"✔️ Generated Directory in google drive -- {dir_path}")
    elif (language == "한국어"):
        print(f"✔️ 구글 드라이브에 폴더를 생성했어요 -- {dir_path}")


import torch
if torch.cuda.is_available():
    print(f"✔️ [Torch cuda is available] Current Torch Version Is: {torch.__version__}")

else :
    print(f"❗ [Torch cuda is Not Available] -- Please Enable GPU! | GPU를 활성화해 주세요! ")
    exit()

print()
if (language == "English"):
    print(f"🧐 Please make sure YOUR 🗂️dataset.zip to be in your Google Drive's 📂MiriVoice/VITS2/train/dataset. ")
elif (language == "한국어"):
    print(f"🧐 구글 드라이브의 📂MiriVoice/VITS2/train/dataset 에 🗂️dataset.zip이 들어가 있는지 잘 확인해 주세요. ")

In [ ]:
#@title 0-2. Choose Finetuning Model | 파인튜닝 모델
#@markdown ⚠️If you want to resume training with your own checkpoints or do not want finetuning, please SKIP this cell. <br> ⚠️보유한 체크포인트를 사용해 이어서 훈련을 진행하려 하거나, 파인튜닝을 하지 않을 계획이라면, 이 셀을 실행할 필요가 없어요.
#@markdown <br>If your voice pitch is low, please choose Rivo, otherwise Miri.
#@markdown <br>본인의 목소리 톤이 낮다면 Rivo를, 아니라면 Miri를 골라 주세요.
import os
fintune_model = "Miri" #@param = ["Miri", "Rivo"]

!apt-get install aria2


if fintune_model == "Miri":
    ! aria2c -x 16 "https://github.com/EX3exp/MiriVoiceSupport-VITS2/releases/latest/download/Base_Miri.zip"
    !unzip -qq '/content/Base_Miri.zip' -d '/content/drive/MyDrive/MiriVoice/VITS2/checkpoints'
elif fintune_model == "Rivo":
    ! aria2c -x 16 "https://github.com/EX3exp/MiriVoiceSupport-VITS2/releases/latest/download/Base_Rivo.zip"
    !unzip -qq '/content/Base_Rivo.zip' -d '/content/drive/MyDrive/MiriVoice/VITS2/checkpoints'




In [ ]:
dataset_zip_name = "dataset.zip" #@param
dataset_zip_path = os.path.join(dir_paths[0], dataset_zip_name)

if (language == "English"):
    print(f"🔸 1. Unzipping dataset ({dataset_zip_path})...")
elif (language == "한국어"):
    print(f"🔸 1. 데이터셋 압축 해제 중 ({dataset_zip_path})...")
!unzip -qq '{dataset_zip_path}' -d '{train_main_path}'

if (language == "English"):
    print(f"✔️ {dataset_zip_name} unzipped in: {train_main_path} ")
elif (language == "한국어"):
    print(f"✔️ {dataset_zip_name}을(를) {train_main_path}에 압축 해제했어요. ")
train_filelist_path = ""
val_filelist_path = ""

config_file_dir = "/content/MiriVoiceSupport-VITS2/configs/"
config_file = "config-mirivoice-colab-fintune-v2.json" #@param ["config-mirivoice-colab-fintune-v2.json", "config-mirivoice-colab-no_fintune.json", "config-mirivoice-colab-fintune.json", "config-mirivoice-colab-enarctic-fintune-v2.json", "config-mirivoice-colab-enarctic-no_fintune.json"]
config_path = f"{config_file_dir}{config_file}"

#@title 2. Set Files | 파일 지정

if (language == "English"):
    print(f"🔸 2. Building monotonic align...")
elif (language == "한국어"):
    print(f"🔸 2. monotonic align 빌드 중...")
os.chdir('/content/MiriVoiceSupport-VITS2/monotonic_align')
os.makedirs('/content/MiriVoiceSupport-VITS2/monotonic_align/monotonic_align', exist_ok=True)
!python setup.py build_ext --inplace

if (language == "English"):
    print(f"✔️ Done!")
elif (language == "한국어"):
    print(f"✔️ 완료! ")


In [ ]:
#@title Audio Preprocess | 오디오 전처리
import os
import librosa
import soundfile as sf

folder_tr_path = '/content/train/train'
folder_vl_path = '/content/train/validation'

length = len(os.listdir(folder_tr_path))

print(f"[   Train set   ]  total: {length-1} downsampling to 44100kHZ ... ")
for i, filename in enumerate(os.listdir(folder_tr_path)):
    if filename.endswith('.wav'):
        file_path = os.path.join(folder_tr_path, filename)

        y, sr = librosa.load(file_path, sr=None)
        y = librosa.util.normalize(y)
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=44100)


        output_file_path = os.path.join(folder_tr_path, filename)

        sf.write(output_file_path, y_resampled, 44100)
        print("*", end="")

print()
length = len(os.listdir(folder_vl_path))
print(f"[Validation set]  total: {length-1} downsampling to 44100kHZ")
for i, filename in enumerate(os.listdir(folder_vl_path)):
    if filename.endswith('.wav'):
        file_path = os.path.join(folder_vl_path, filename)

        y, sr = librosa.load(file_path, sr=None)
        y = librosa.util.normalize(y)
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=44100)

        output_file_path = os.path.join(folder_vl_path, filename)

        sf.write(output_file_path, y_resampled, 44100)
        print("*", end="")




In [ ]:
#@title Tensorboard | 텐서보드

if (language == "English"):
    print("--- TensorBoard ---")

    print()
    print("Please Run cell right below... ")
    print("All you have to do is watching TensorBoard at here, And stop training in proper time. (to stop training, press stop button in left of below cell.)")
    print()
    print(f"🧐 Please check scalars' ❇️mel_loss, ❇️kl_loss during training.")
    print(f"🧐 You might can stop training when: ")
    print(f"\t- mel_loss is smaller than 15")
    print(f"\t- kl_loss is almost 1 or 2")

elif (language == "한국어"):
    print("--- 텐서보드 ---")

    print()
    print("이제 이 셀 아래의 셀을 실행해 주세요... ")
    print("여기서 텐서보드를 계속 지켜봐 주세요. 적정한 때에 하단 셀 왼쪽의 정지 버튼을 눌러 훈련을 중단해 주시면 됩니다.")
    print()
    print(f"🧐 훈련이 이루어지는 동안, scalars의 ❇️mel_loss, ❇️kl_loss 에 주목하세요.")
    print(f"🧐 훈련을 멈춰도 될 때: ")
    print(f"\t- mel_loss가 15 이하일 때")
    print(f"\t- kl_loss가 1~2 가량일 때")


%load_ext tensorboard
%tensorboard --logdir '{train_main_path}/logs'


In [ ]:
#@title Training | 훈련
import os
os.chdir(train_main_path)
!python /content/MiriVoiceSupport-VITS2/train_ms.py -c '{config_path}' -m mirivoice-vits2